In [2]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/eb/f1/828bbccc84a72db960a7d116f55f3f6aec9f5658f5d32ce9db20142d5742/tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.18.0 from https://files.pythonhosted.org/packages/ae/4e/44ce609139065035c56fe570fe7f0ee8d06180c99a424bac588472052c5d/tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2db


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ------------ ------------------------- 125.4/390.3 MB 980.0 kB/s eta 0:04:31
   ------------ ------------------------- 125.4/390.3 MB 972.8 kB/s eta 0:04:33
   ------------ ------------------------- 125.5/390.3 MB 970.0 kB/s eta 0:04:33
   ------------ ------------------------- 125.5/390.3 MB 970.0 kB/s eta 0:04:33
   ------------ ------------------------- 125.5/390.3 MB 970.0 kB/s eta 0:04:33
   ------------ ------------------------- 125.5/390.3 MB 970.0 kB/s eta 0:04:33
   ------------ ------------------------- 125.5/390.3 MB 957.2 kB/s eta 0:04:37
   ------------ ------------------------- 125.5/390.3 MB 961.4 kB/s eta 0:04:36
   ------------ ------------------------- 125.5/390.3 MB 958.6 kB/s eta 0:04:37
   ------------ ------------------------- 125.6/390.3 MB 961.4 kB/s eta 0:04:36
   ------------ ------------------------- 125.6/390.3 MB 960.0 kB/s eta 0:04:36
   ------------ ------------------------- 125.7/390.3 MB 955.8 kB/s eta 0:04:37
   ------------ -----------------------